# day 29 
## < bayes' theorem >


## Basic Understaing for Bayes'Theorem

In [6]:
# 기본 이해

# 홍길동 받은 메일의 80 - 스팸메일
# 스팸 95 - 대출
# 정상 2 - 대출

# 문제) 
# 대출 이라는 단어가 제목에서 발견, 스팸확률

# 풀이)
# P(스팸 | 대출) = P(대출|스팸) * P(스팸) / P(대출)
#                = 0.95 * 0.8 / P(대출)
# P(A|B) = A
# P(대출) = P(대출|스팸) P(스팸) + P(대출|정상) P(정상)
#         = 0.95 * 0.8 + 0.02 * 0.2 = 0.764
    
# P(스팸|대출) = 0.95 * 0.8 / 0. 764

In [7]:

# 프로젝트
# 메일 스팸/정상 분류기
# ( + RNN  )

# 1)
# 종속적 접근 ( 문맥 )
# 문장 => 확률
# P('나는, 지금, 몹시, 배가, 고파요')
# 조건부확률 베이즈 이론 
# '나는' 다음 뭐가 나오나 

# 2) 
# Count(나는 지금 몹시 배가 고파요 ) / => 10
# Count(나는 지금 몹시 배가 ) / ========> 100
# 검색 추천 시스템
# 이제까지 사람들이 많이 이용한 단어 추천

# 모수가 대단히 작거나 0 일수도있다
# => Ngram 을 통해 해당 문제점 탈피 할수 있다 (UNI, BI, TRI) Gram


## Ngram 접근방식 이용한 단어 추천


In [8]:
# Ngram 접근방식 이용한 단어 추천
# 앞단어 하나만 보겠다 
# N = 1 일때, 마지막요소 참조 
# N = 2 , EX) 고파요 | (몹시, 배가)

# 한계 1) 해당 N그램과 달리 중요의미를 지닌 단어가 
# 범위 이외에 자리한다면 해단 단어를 잡지 못하고 간다
# N 이 작으면 작을수록 모델이 양질이라 볼수 있다 
# Out Of Voca 

# 한계 2) N이 너무 작으면 문맥의 정확함을 파악하기 어려워진다  
# EX) 너는, 나는 의미 차이 파악 불가
# SOL) RNN - WORD2VEC 


# Code Parts

In [ ]:
# TF - IDF
# Bag of Words ( BOW )
# : 단어들의 등장 횟수로 표현 
# 1. 주어진 단어에 대해 고유의 인덱스를 부여한다 
# 2. 단어의 등장 횟수 벡터생성 


In [10]:
from konlpy.tag import Okt
import re

okt = Okt()
token = "오늘은 금요일입니다. 내일은 토요일입니다. 다음주 화요일에는 특강이 있습니다."
# token에 저장된 문자중에서 (.)을 제거하세요
token = re.sub("\.","",token)
token


'오늘은 금요일입니다 내일은 토요일입니다 다음주 화요일에는 특강이 있습니다'

In [11]:
okt.morphs(token)

['오늘',
 '은',
 '금요일',
 '입니다',
 '내일',
 '은',
 '토요일',
 '입니다',
 '다음주',
 '화요일',
 '에는',
 '특강',
 '이',
 '있습니다']

In [12]:
# 1. 주어진 단어에 대해 고유의 인덱스를 부여한다 
len(okt.morphs(token)) # (0-13)
# '은', '입니다' 중복되므로 => 0 ~ 11 인덱싱


14

In [ ]:
# 2. 단어의 등장 횟수 벡터생성 
# {'오늘' : 0,.....'있습니다':11}
# [1,2,1,1,1....]

In [18]:
from nltk import FreqDist
count=FreqDist(okt.morphs(token))
count

FreqDist({'은': 2, '입니다': 2, '오늘': 1, '금요일': 1, '내일': 1, '토요일': 1, '다음주': 1, '화요일': 1, '에는': 1, '특강': 1, ...})

In [20]:
# from keras_preprocessing.text import Tokenizer
# tok = Tokenizer()
# tok.fit_on_texts([token])
# tok.word_counts
# tok.word_index # 인덱싱

OrderedDict([('오늘은', 1),
             ('금요일입니다', 1),
             ('내일은', 1),
             ('토요일입니다', 1),
             ('다음주', 1),
             ('화요일에는', 1),
             ('특강이', 1),
             ('있습니다', 1)])

In [22]:
word2index = {}
bow = []
for voc in token:
    if voc not in word2index.keys():
            word2index[voc] = len(word2index)
            bow.insert(len(word2index)-1, 1)
    else:
            index = word2index.get(voc)
            bow[index] = bow[index] + 1
print(word2index)

{'오': 0, '늘': 1, '은': 2, ' ': 3, '금': 4, '요': 5, '일': 6, '입': 7, '니': 8, '다': 9, '내': 10, '토': 11, '음': 12, '주': 13, '화': 14, '에': 15, '는': 16, '특': 17, '강': 18, '이': 19, '있': 20, '습': 21}


In [23]:
bow

[1, 1, 2, 7, 1, 3, 4, 2, 3, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
text = ['you know I want your love. because I love you.']
# vec = CountVectorizer()
vec = CountVectorizer(stop_words=['|']) # 불용어 사용 (사용자정의)
vec = CountVectorizer(stop_words=['english']) # 불용어 사용 (사용자정의)
print(vec.fit_transform(text).toarray()) # 각 단어별 개수를 세며, 변환 
print(vec.vocabulary_)
# = > {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}
# 각각의 숫자는 해당 인덱스 문자를 뜻혀
# countVectorizer = BOW 생성 클래스 


[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [30]:
# 문장간 유사도 조사 
# ( 1 - target ) 오늘 역삼에서 맛있는 돈가스를 먹었다
# ( 2 - source ) 역삼에서 먹었던 오늘의 돈가스는 맛있었다. 

# 2번 원문에 대해 1번 대상문장이 어느정도 유사한지 출력
# 1번 문자에 대해 n=2 로 하여 문장 분리하면 
# => '오늘','늘 ',' 역','역삼',...'다.'
# => 만약 길이가 20 이라 가정하고, 
# 2번 원문에 대해서도 n = 2로 하여 문장을 분리할시, 공통으로 존재하는 단어 5개
# 존재한다면 , 5/20 = 25% 유사도를 갖는다 

# 두 문장의 유사도? (By Using N-gRAM)
# ex) n = 2
# (1) '오늘','늘 ',' 역','역삼',...'다.' # 두글자씩 끊는다
# ex) 20개 중 5개가 (2)과 일치 
# (2) '역삼','삼에','에서',...'다.'



In [ ]:
# n( 2 ) 정도로...이 커지면 커질수록 유사도는 떨어진다 
# 토크나이즈 짜르고
# 불용어 날리고 
# 형태소 분리 / 명사가지고 비교


In [ ]:
# TF(단어 빈도, term frequency)는 특정한 단어가 문서 내에 
# 얼마나 자주 등장하는지를 나타내는 값.
# 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다.
# TF : 문서 D에서 단어 T의 등장 횟수


In [38]:
corpus = ['you know I want your love',
          'I like you',
          'what i should i do']
#DTM 생성
vec=CountVectorizer()
print(vec.fit_transform(corpus).toarray())
print(vec.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [39]:
# DTM 생성
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv=TfidfVectorizer().fit(corpus)
print(tfidfv)
print('='*50)
print(tfidfv.transform(corpus).toarray())
print('='*50)
print(tfidfv.vocabulary_)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [51]:
from sklearn.datasets import fetch_20newsgroups
# 일종의 글이 있고 어떠한 분류에 의한 글인가 

newsdata = fetch_20newsgroups(subset='train')
# train / test data 나눠져 있다
# 11400 개 / 7500 개 

print(newsdata.keys())
print(len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names))
print(newsdata.target_names)
# 20 가지의 노드 ... 
print(newsdata.target[0])
print(newsdata.target_names[7])
print(newsdata.data[1]) # newsdata.data = 뉴스들이 담겨있따 
# 20개의 카테고리 중에서도 rec.autos 에 해당하는 글이다
print(newsdata.target[1]) # 이문서의 분류가 어떤 분류냐 / 분류의 번호
print(newsdata.target_names[4]) # 분류의 이름은?

# 일종의 글이 있고 어떠한 분류에 의한 글인가 

# => 빈도표와 우도표의 mixture 표 도출 
# => 사후확률이 도출되어진다

# # 11314 행 - 문서, 열 - 단어
# sklearn lib.bayes package _ a various of classifier pick! 
# fit 하면, 분류기 생성
# 그 다음 test 한다 
# laplace : default = 1 


dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
11314 11314 20
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
7
rec.autos
From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated s

In [52]:
# future image output
# P(분류 1 | TEST DOC 1 ) = 사후확률? ===> max
# .
# .
# .
# P(분류 20 | TEST DOC 1 ) = 사후확률

In [55]:
# 1. 텍스트 -> bow 생성
dtmVector = CountVectorizer()
xtrainDtm = dtmVector.fit_transform(newsdata.data)
xtrainDtm.shape
# 여기등장하지않은건 oov 이다 
# bow에 없다 
# TF-IDF 행렬을 이용해서 분류하면 정확도 높여진다
# DTM => TF-IDF 변환 성능 더욱 고성능화


(11314, 130107)

In [58]:
# 2. DTM -> TF-IDF 행렬로 변환
# 위의 TF-IDF 바로 생성 하던 클래스와 다르다
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(xtrainDtm)


In [61]:
# 3. 나이브베이즈 분류기 모델 생성
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(tfidfv, newsdata.target) 
# fit 에 data 만 너겨주면 끝나 # (train data, newsdata.target) #답이 뭔지도 알려줘야해
# alpha : raplace smooth default value
# 모델 완성했으니 테스트 데이터넣어 테스트 레이블과 비교하여 해당 acc 도출


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [62]:
testdata = fetch_20newsgroups(subset='test',shuffle=True)
xtestDtm = dtmVector.transform(testdata.data)
tfidf_test = tfidf_transformer.transform(xtestDtm)
# Dtm으로 변환 


In [63]:
from sklearn.metrics import accuracy_score
predicted =  model.predict(tfidf_test)
print("정확도:",accuracy_score(testdata.target, predicted))
# 토픽 모델링  # 주제어찾기  

정확도: 0.7738980350504514


In [ ]:
# 문제 

"""
1. 스팸필터
-홍길동 받은 메일의 80% 스팸메일.
-스팸메일의 95%에서 '대출'이라는 단어
-정상메일의 2%에서 '대출' 단어
---------주어진 데이터--------
문제)방금 메일을 받음.
'대출'이라는 단어가 제목에서 발견됨.
스팸일 확률?
P(스팸|대출)=P(대출|스팸)P(스팸) / P(대출)
                = 0.95*0.8 / P(대출)

P(대출)=P(대출|스팸)P(스팸)+P(대출|정상)P(정상)
=             0.95 * 0.8 + 0.02 * 0.2 = 0.764

P(스팸|대출)= 0.95*0.8 / 0.764 =???


2. 문장간 유사도 조사

(1, target) 
오늘 역삼에서 맛있는 돈가스를 먹었다.
(2, source) 
역삼에서 먹었던 오늘의 돈가스는 맛있었다.


(2)번 원문장에 대해 (1)번 대상문장이 어느정도 유사한지 출력?
(1)번 문장에 대해 n=2로 하여 문장 분리하면
=> '오늘','늘 ',' 역','역삼', ... '다.'
=>만약 길이가 20이라 가정하고, 
(2)번 원문장에 대해서도 n=2로 하여 문장 분리했을때,
공통으로 존재하는 단어가 5개 존재한다면,
5/20 = 25%의 유사도를 갖는다.

두 문장의 유사도?(using n-gram)
ex) n=2
(1) '오늘','늘 ',' 역','역삼', ... '다.'  
=> 20개 중 5개가 (2)과 일치
(2) '역삼', '삼에', '에서', ...'다.'
"""
